### Setup

In [22]:
import openai, os, requests
from dotenv import load_dotenv
import json

load_dotenv()

openai.api_type = "azure"
# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version = os.getenv("OPENAI_API_VERSION")
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_key = os.getenv("OPENAI_API_KEY") # Add your OpenAI API key here
deployment_id = "gpt-4-32k" # Add your deployment ID here

# Azure AI Search setup
search_endpoint = "https://cog-search-aia.search.windows.net"; # Add your Azure AI Search endpoint here
search_key = os.getenv("VECTOR_STORE_KEY"); # Add your Azure AI Search admin key here
search_index_name = "pythonsample-documents-chunk-index"; # Add your Azure AI Search index name here

def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.

    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    openai.requestssession = session

def run_chat_completion(message_text):
    completion = openai.ChatCompletion.create(
    messages=message_text,
    deployment_id=deployment_id,
    dataSources=[  # camelCase is intentional, as this is the format the API expects
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": search_endpoint,
                "key": search_key,
                "indexName": search_index_name,
                "semanticConfiguration": "default",
                "queryType": "semantic",
                "inScope": True,
                "roleInformation": "Responda as perguntas em Português",
                "strictness": 3,
            }
        }
    ]
    )
    return completion

setup_byod(deployment_id)

### Primeira pergunta do usuário

In [27]:
message_text = [{"role": "user", "content": "List 3 job roles and a brief description for each one"}]

completion = run_chat_completion(message_text)

completion['choices'][0]['message']['content']

'1. **Gerente de Recursos Humanos**: Este papel é responsável por supervisionar o recrutamento, desenvolvimento e retenção de funcionários em todos os níveis da empresa. O Gerente Sênior de Recursos Humanos fornece liderança e direção para a equipe de Recursos Humanos, garantindo que todos os processos e procedimentos sejam seguidos e que o departamento atinja seus objetivos e metas[doc4].\n\n2. **Gerente de Pesquisa e Desenvolvimento**: O Gerente de Pesquisa e Desenvolvimento da Contoso Electronics cria e lidera as atividades de pesquisa e desenvolvimento da empresa. Este cargo é responsável por projetar e implementar soluções inovadoras para maximizar o desempenho do produto e otimizar a satisfação do cliente[doc1].\n\n3. **Vice-Presidente de Operações**: O Vice-Presidente de Operações da Contoso Electronics é responsável por supervisionar as operações diárias do negócio e liderar a equipe de operações. O VP de Operações é responsável por desenvolver e executar planos estratégicos pa

In [28]:
content_string = completion['choices'][0]['message']['context']['messages'][0]['content']

# Load the string as a JSON object
content_json = json.loads(content_string)

# Pretty print the JSON object
pretty_content = json.dumps(content_json, indent=4)
print(pretty_content)

{
    "citations": [
        {
            "content": ". \n\n\u2022 Oversee the recruitment and selection process, ensuring that hiring and promotion \n\ndecisions are made in compliance with applicable laws and regulations. \n\n\u2022 Monitor employee performance, providing feedback and coaching as necessary. \n\n\u2022 Develop compensation and benefit strategies to attract and retain top talent. \n\n\u2022 Handle employee relations issues such as disciplinary actions, grievances, and \n\nperformance management. \n\n\u2022 Ensure compliance with all applicable labor laws and regulations. \n\n\u2022 Develop and maintain relationships with external vendors and service providers. \n\n\u2022 Monitor and analyze employee engagement and satisfaction. \n\n\u2022 Ensure a safe and healthy work environment. \n\npageimage26.jpg\n\n\n\n\n\n\u2022 Lead the Human Resources department in a manner that supports and guides the \n\nachievement of the overall business objectives of the organization. \n

### Follow up Question

In [29]:
message_text.append({"role": "assistant", "content": completion['choices'][0]['message']['content']})
message_text.append({"role": "user", "content": "Tell me a little bit more about the second one"})
message_text

[{'role': 'user',
  'content': 'List 3 job roles and a brief description for each one'},
 {'role': 'assistant',
  'content': '1. **Gerente de Recursos Humanos**: Este papel é responsável por supervisionar o recrutamento, desenvolvimento e retenção de funcionários em todos os níveis da empresa. O Gerente Sênior de Recursos Humanos fornece liderança e direção para a equipe de Recursos Humanos, garantindo que todos os processos e procedimentos sejam seguidos e que o departamento atinja seus objetivos e metas[doc4].\n\n2. **Gerente de Pesquisa e Desenvolvimento**: O Gerente de Pesquisa e Desenvolvimento da Contoso Electronics cria e lidera as atividades de pesquisa e desenvolvimento da empresa. Este cargo é responsável por projetar e implementar soluções inovadoras para maximizar o desempenho do produto e otimizar a satisfação do cliente[doc1].\n\n3. **Vice-Presidente de Operações**: O Vice-Presidente de Operações da Contoso Electronics é responsável por supervisionar as operações diárias 

In [30]:
completion = run_chat_completion(message_text)

completion['choices'][0]['message']['content']

'O Gerente de Pesquisa e Desenvolvimento da Contoso Electronics é responsável por criar e liderar as atividades de pesquisa e desenvolvimento da empresa. Este papel envolve a concepção e implementação de soluções inovadoras para maximizar o desempenho do produto e otimizar a satisfação do cliente. As responsabilidades incluem o desenvolvimento e implementação de estratégias para pesquisa e desenvolvimento, o design, teste e otimização de produtos para a satisfação ideal do cliente, e a gestão de uma equipe de pesquisadores e engenheiros para criar produtos inovadores. Além disso, o Gerente de Pesquisa e Desenvolvimento monitora o processo de desenvolvimento para garantir que os prazos sejam cumpridos, analisa o feedback do cliente e faz as alterações necessárias nos designs dos produtos[doc1].'

In [31]:
content_string = completion['choices'][0]['message']['context']['messages'][0]['content']

# Load the string as a JSON object
content_json = json.loads(content_string)

# Pretty print the JSON object
pretty_content = json.dumps(content_json, indent=4)
print(pretty_content)

{
    "citations": [
        {
            "content": ". \n\n\u2022 Oversee the recruitment and selection process, ensuring that hiring and promotion \n\ndecisions are made in compliance with applicable laws and regulations. \n\n\u2022 Monitor employee performance, providing feedback and coaching as necessary. \n\n\u2022 Develop compensation and benefit strategies to attract and retain top talent. \n\n\u2022 Handle employee relations issues such as disciplinary actions, grievances, and \n\nperformance management. \n\n\u2022 Ensure compliance with all applicable labor laws and regulations. \n\n\u2022 Develop and maintain relationships with external vendors and service providers. \n\n\u2022 Monitor and analyze employee engagement and satisfaction. \n\n\u2022 Ensure a safe and healthy work environment. \n\npageimage26.jpg\n\n\n\n\n\n\u2022 Lead the Human Resources department in a manner that supports and guides the \n\nachievement of the overall business objectives of the organization. \n

In [32]:
message_text.append({"role": "assistant", "content": completion['choices'][0]['message']['content']})
message_text.append({"role": "user", "content": "Now tell me a little bit more about the third one"})
message_text

[{'role': 'user',
  'content': 'List 3 job roles and a brief description for each one'},
 {'role': 'assistant',
  'content': '1. **Gerente de Recursos Humanos**: Este papel é responsável por supervisionar o recrutamento, desenvolvimento e retenção de funcionários em todos os níveis da empresa. O Gerente Sênior de Recursos Humanos fornece liderança e direção para a equipe de Recursos Humanos, garantindo que todos os processos e procedimentos sejam seguidos e que o departamento atinja seus objetivos e metas[doc4].\n\n2. **Gerente de Pesquisa e Desenvolvimento**: O Gerente de Pesquisa e Desenvolvimento da Contoso Electronics cria e lidera as atividades de pesquisa e desenvolvimento da empresa. Este cargo é responsável por projetar e implementar soluções inovadoras para maximizar o desempenho do produto e otimizar a satisfação do cliente[doc1].\n\n3. **Vice-Presidente de Operações**: O Vice-Presidente de Operações da Contoso Electronics é responsável por supervisionar as operações diárias 

In [34]:
completion = run_chat_completion(message_text)

completion['choices'][0]['message']['content']

'O Vice-Presidente de Operações da Contoso Electronics é responsável por supervisionar as operações diárias do negócio e liderar a equipe de operações. Este papel envolve o desenvolvimento de metas e objetivos de curto e longo prazo para garantir o sucesso organizacional. O VP de Operações monitora o desempenho operacional para garantir a conformidade com os padrões estabelecidos, estabelece e mantém fortes relações de trabalho com as partes interessadas internas e externas, e lidera o desenvolvimento de políticas e procedimentos operacionais. Além disso, o VP de Operações identifica e implementa melhorias de processo para aumentar a eficiência e garante que os padrões de segurança e regulamentos legais sejam cumpridos. Este papel requer um diploma de bacharel em negócios, operações ou um campo relacionado, e mais de 10 anos de experiência em gestão de operações e/ou operações de negócios[doc1].'

In [35]:
content_string = completion['choices'][0]['message']['context']['messages'][0]['content']

# Load the string as a JSON object
content_json = json.loads(content_string)

# Pretty print the JSON object
pretty_content = json.dumps(content_json, indent=4)
print(pretty_content)

{
    "citations": [
        {
            "content": ".  \n\n\u2022 Monitor operational performance to ensure compliance with established standards.  \n\n\u2022 Establish and maintain strong working relationships with internal and external \n\nstakeholders.  \n\n\u2022 Lead the development of operational policies and procedures.  \n\n\u2022 Develop short- and long-term goals and objectives to ensure organizational success.  \n\n\u2022 Identify and implement process improvements to increase efficiency.  \n\n\u2022 Ensure safety standards and legal regulations are met.  \n\n\u2022 Stay up-to-date with industry trends and best practices.  \n\n \n\nQualifications:  \n\n\u2022 Bachelor\u2019s degree in business, operations, or a related field.  \n\n\u2022 10+ years of experience in operations management and/or business operations.  \n\n\u2022 Proven track record of successfully leading operational teams and achieving \n\norganizational goals.  \n\n\u2022 Excellent problem-solving and decis